In [1]:
!pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import sys
import urllib.request
import os
import pymysql

import time
import pandas as pd
import mysql.connector
import time

In [3]:
wd = webdriver.Chrome()

In [4]:
# 국내 모델

data_list = []

for i in range(2022,2025):
    # 2022 ~ 2024
    for j in range(1,13):
        # 2022-01 ~ 2024-08까지만 출력
        if i == 2024 and j == 9:
            break;
        
        wd.get(f'https://auto.danawa.com/auto/?Work=record&Tab=Model&Brand=303,307,304,312,326,321&Month={i}-{j:02}-00&MonthTo=')
        time.sleep(2)
        
        element = wd.find_element(By.CSS_SELECTOR, '#autodanawa_gridC > div.gridMain > article > main > div > table.recordTable.model > tbody')
        
        rows = element.find_elements(By.TAG_NAME, 'tr')
        
        sales_month = f'{i}-{j:02}'
        print(sales_month)
        for row in rows:
            
            # 다른 class 속성이 있는 경우 건너뜀 - <tr>에 대해서만 작동
            class_attribute = row.get_attribute('class')
            if class_attribute and ('sub' in class_attribute or 'model_' in class_attribute):
                continue  

            # 판매 순위
            rank = row.find_element(By.CLASS_NAME, 'rank').text
            
            # 모델명
            model_element = row.find_element(By.CSS_SELECTOR, 'td.title a')
            model_name = model_element.text.strip()  # 모델명
            
            # 판매량
            sales = row.find_element(By.CLASS_NAME, 'num').text.split(' ')[0].replace(',', '')
            
            # 점유율
            market_share = row.find_element(By.CLASS_NAME, 'rate').text.replace('%', '')
            
            # 전월 대비 판매량 추출
            format_change = lambda text: '-' + text.split('▼')[0].strip() if '▼' in text else text.split('▲')[0].strip()
            last_month_values = row.find_elements(By.CLASS_NAME, 'right')[1].text.split('\n')
            
            last_month_sales = last_month_values[0].replace(',', '') # 전월 판매량
            last_month_change = format_change(last_month_values[1]).replace(',', '') # 전월 대비 증감
            
            data_dict = {
                '순위': rank,
                '모델명': model_name,
                '판매량': sales,
                '점유율': market_share,
                '전월 판매량': last_month_sales,
                '전월 대비 증감': last_month_change,
                '판매월' : sales_month
            }
            print(data_dict)
            data_list.append(data_dict)

2022-01
{'순위': '1', '모델명': 'G80', '판매량': '5501', '점유율': '5.9', '전월 판매량': '6194', '전월 대비 증감': '-693', '판매월': '2022-01'}
{'순위': '2', '모델명': '포터2', '판매량': '5443', '점유율': '5.8', '전월 판매량': '7633', '전월 대비 증감': '-2190', '판매월': '2022-01'}
{'순위': '3', '모델명': '아반떼', '판매량': '5099', '점유율': '5.4', '전월 판매량': '6767', '전월 대비 증감': '-1668', '판매월': '2022-01'}
{'순위': '4', '모델명': '쏘렌토', '판매량': '5066', '점유율': '5.4', '전월 판매량': '5561', '전월 대비 증감': '-495', '판매월': '2022-01'}
{'순위': '5', '모델명': '봉고 3', '판매량': '4643', '점유율': '4.9', '전월 판매량': '4553', '전월 대비 증감': '90', '판매월': '2022-01'}
{'순위': '6', '모델명': '스포티지', '판매량': '4455', '점유율': '4.7', '전월 판매량': '7442', '전월 대비 증감': '-2987', '판매월': '2022-01'}
{'순위': '7', '모델명': '팰리세이드', '판매량': '4302', '점유율': '4.6', '전월 판매량': '3716', '전월 대비 증감': '586', '판매월': '2022-01'}
{'순위': '8', '모델명': '카니발', '판매량': '4114', '점유율': '4.4', '전월 판매량': '5619', '전월 대비 증감': '-1505', '판매월': '2022-01'}
{'순위': '9', '모델명': '캐스퍼', '판매량': '3948', '점유율': '4.2', '전월 판매량': '4127', '전월 대비 증감': '-179', '판매월':

In [5]:
df = pd.DataFrame(data_list)
df

,순위,모델명,판매량,점유율,전월 판매량,전월 대비 증감,판매월
0,1,G80,5501,5.9,6194,-693,2022-01
1,2,포터2,5443,5.8,7633,-2190,2022-01
2,3,아반떼,5099,5.4,6767,-1668,2022-01
3,4,쏘렌토,5066,5.4,5561,-495,2022-01
4,5,봉고 3,4643,4.9,4553,90,2022-01
...,...,...,...,...,...,...,...
2190,66,GV60,33,0.0,64,-31,2024-08
2191,67,마스터,20,0.0,20,-0,2024-08
2192,68,타호,9,0.0,16,-7,2024-08
2193,69,Electrified G80,3,0.0,33,-30,2024-08


In [6]:
import mysql.connector

# 데이터베이스 연결 설정
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'database': 'pjt01'
}


# 데이터베이스 연결
try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    for entry in data_list:
        model_name = entry['모델명']
        
        # 모델명을 통해 vehicle_id 가져오기
        cursor.execute("SELECT vehicle_id FROM vehicles WHERE vehicle_name = %s", (model_name,))
        vehicle_id = cursor.fetchone()
        
        if vehicle_id:
            vehicle_id = vehicle_id[0]  # 첫 번째 요소 가져오기
            
            # 데이터 삽입
            cursor.execute("""
                INSERT INTO domestic_model_sales (vehicle_id, sales_rank, market_share, last_month_sales, last_month_change, sales_month) 
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, (vehicle_id, entry['순위'], entry['판매량'], entry['점유율'], entry['전월 판매량'], entry['전월 대비 증감'], entry['판매월']+'-01'))
        else:
            print(f"{model_name}에 대한 vehicle_id를 찾을 수 없습니다.")

    # 변경 사항 커밋
    conn.commit()
    print("데이터가 성공적으로 삽입되었습니다.")

except mysql.connector.Error as err:
    print(f"오류 발생: {err}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


오류 발생: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'rank, sales_rank, market_share, last_month_sales, last_month_change, sales_month' at line 1


In [ ]:
# 해외 모델

data_list = []

# domestic_vehicels_numbers = [303, 307, 304, 326, 321, 322, 312]
# international_vehicles_numbers =[362,349,371,376,459,611,367,381,486,491]

for i in range(2022,2025):
    # 2022 ~ 2024
    for j in range(1,13):
        # 2022-01 ~ 2024-07까지만 출력
        if i == 2024 and j == 8:
            break;
        
        wd.get(f'https://auto.danawa.com/auto/?Work=record&Tab=Model&Brand=362,349,371,611,459,376,491,486,381,367&Month={i}-{j:02}-00&MonthTo=')
        time.sleep(2)
        
        element = wd.find_element(By.CSS_SELECTOR, '#autodanawa_gridC > div.gridMain > article > main > div > table.recordTable.model > tbody')
        #autodanawa_gridC > div.gridMain > article > main > div:nth-child(1) > table > tbody > tr:nth-child(1) > td.rank
        
        rows = element.find_elements(By.TAG_NAME, 'tr')
        
        sales_month = f'{i}-{j:02}'
        print(sales_month)
        for row in rows:
            
            # 다른 class 속성이 있는 경우 건너뜀 - <tr>에 대해서만 작동
            class_attribute = row.get_attribute('class')
            if class_attribute and ('sub' in class_attribute or 'model_' in class_attribute):
                continue
            
            # 판매 순위
            rank = row.find_element(By.CLASS_NAME, 'rank').text
            
            # 모델명
            model_element = row.find_element(By.CSS_SELECTOR, 'td.title a')
            model_name = model_element.text.strip()  # 모델명
            
            # 판매량
            sales = row.find_element(By.CLASS_NAME, 'num').text.split(' ')[0].replace(',', '')
            
            # 점유율
            market_share = row.find_element(By.CLASS_NAME, 'rate').text.replace('%', '')
            
            # 전월 대비 판매량 추출
            format_change = lambda text: '-' + text.split('▼')[0].strip() if '▼' in text else text.split('▲')[0].strip()
            last_month_values = row.find_elements(By.CLASS_NAME, 'right')[1].text.split('\n')
            
            last_month_sales = last_month_values[0].replace(',', '') # 전월 판매량
            last_month_change = format_change(last_month_values[1]).replace(',', '') # 전월 대비 증감
            
            data_dict = {
                '순위': rank,
                '모델명': model_name,
                '판매량': sales,
                '점유율': market_share,
                '전월 판매량': last_month_sales,
                '전월 대비 증감': last_month_change,
                '판매월' : sales_month
            }
            print(data_dict)
            data_list.append(data_dict)

2022-01
{'순위': '1', '모델명': '5 Series', '판매량': '1997', '점유율': '13.5', '전월 판매량': '1130', '전월 대비 증감': '867', '판매월': '2022-01'}
{'순위': '2', '모델명': 'E-Class', '판매량': '1884', '점유율': '12.8', '전월 판매량': '2776', '전월 대비 증감': '-892', '판매월': '2022-01'}
{'순위': '3', '모델명': 'S-Class', '판매량': '927', '점유율': '6.3', '전월 판매량': '1818', '전월 대비 증감': '-891', '판매월': '2022-01'}
{'순위': '4', '모델명': '3 Series', '판매량': '550', '점유율': '3.7', '전월 판매량': '433', '전월 대비 증감': '117', '판매월': '2022-01'}
{'순위': '5', '모델명': 'X5', '판매량': '505', '점유율': '3.4', '전월 판매량': '444', '전월 대비 증감': '61', '판매월': '2022-01'}
{'순위': '6', '모델명': 'A6', '판매량': '423', '점유율': '2.9', '전월 판매량': '2429', '전월 대비 증감': '-2006', '판매월': '2022-01'}
{'순위': '7', '모델명': 'ES', '판매량': '407', '점유율': '2.8', '전월 판매량': '632', '전월 대비 증감': '-225', '판매월': '2022-01'}
{'순위': '8', '모델명': 'X7', '판매량': '405', '점유율': '2.7', '전월 판매량': '332', '전월 대비 증감': '73', '판매월': '2022-01'}
{'순위': '9', '모델명': 'Hatch', '판매량': '384', '점유율': '2.6', '전월 판매량': '344', '전월 대비 증감': '40', '판매월': '2022

In [ ]:
df = pd.DataFrame(data_list)
df

In [ ]:
import mysql.connector

# 데이터베이스 연결 설정
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'database': 'pjt01'
}


# 데이터베이스 연결
try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    for entry in data_list:
        model_name = entry['모델명']
        
        # 모델명을 통해 vehicle_id 가져오기
        cursor.execute("SELECT vehicle_id FROM vehicles WHERE vehicle_name = %s", (model_name,))
        vehicle_id = cursor.fetchone()
        
        if vehicle_id:
            vehicle_id = vehicle_id[0]  # 첫 번째 요소 가져오기
            
            # 데이터 삽입
            cursor.execute("""
                INSERT INTO international_model_sales (vehicle_id, sales_rank, market_share, last_month_sales, last_month_change, sales_month) 
                VALUES (%s, %s, %s, %s, %s, %s)
            """, (vehicle_id, entry['순위'], entry['판매량']entry['판매량'], entry['점유율'], entry['전월 판매량'], entry['전월 대비 증감'], entry['판매월']+'-01'))
        else:
            print(f"{model_name}에 대한 vehicle_id를 찾을 수 없습니다.")

    # 변경 사항 커밋
    conn.commit()
    print("데이터가 성공적으로 삽입되었습니다.")

except mysql.connector.Error as err:
    print(f"오류 발생: {err}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
